In [14]:
!rm -rf .git .dvc iris.csv.dvc artifacts __pycache__ .ipynb_checkpoints .gitignore dvc.yaml dvc.lock

In [15]:
!pip install -q dvc[gs] scikit-learn pandas joblib
!pip install -q --upgrade google-cloud-aiplatform

In [16]:
!git init
!dvc init
!git config user.email "tarunarora6029@email.com"
!git config user.name "tarunarora6029"

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/jupyter/practice/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                    

In [17]:
from google.cloud import aiplatform

PROJECT_ID = "verdant-nova-461606-f4"
LOCATION = "us-central1"
BUCKET_URI = "gs://ibdpractice"
DVC_REMOTE = "gcsremote"

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)
!dvc remote add -d {DVC_REMOTE} {BUCKET_URI}/dvcstore

Setting 'gcsremote' as a default remote.


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

df = pd.read_csv("iris.csv")
train, test = train_test_split(df, test_size=0.4, stratify=df['species'], random_state=42)

X_train = train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_train = train['species']
X_test = test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_test = test['species']

model = DecisionTreeClassifier(max_depth=3, random_state=1)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, preds))

Accuracy: 0.9833333333333333


In [19]:
import joblib
import os

os.makedirs("artifacts", exist_ok=True)
joblib.dump(model, "artifacts/model.joblib")

['artifacts/model.joblib']

In [20]:
# Track dataset
!dvc add iris.csv
!echo '!iris.csv.dvc' >> .gitignore

# Track model
!dvc add artifacts/model.joblib
!echo '!artifacts/**.dvc' >> .gitignore

# Stage all
!git add .gitignore iris.csv.dvc artifacts/model.joblib.dvc
!git commit -m "Initial commit: iris + model"
!dvc push

 ⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jupyter/practice/.dvc/cache/files/md5'| |0/? [00:0
                                                                                
!
  0%|          |Adding iris.csv to cache              0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/jupyter/practice/ir0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 17.66file/s]

To track the changes with git, run:

	git add iris.csv.dvc .gitignore

To enable auto staging, run:

	dvc config core.autostage true
 ⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Che

In [ ]:
!git branch -m main
!git push -u origin main --force